In [1]:
#Import all required libraries
import numpy as np
import os
from scipy.misc import imread, imresize, imshow
import datetime
import os
import warnings
from PIL import Image 
warnings.filterwarnings("ignore")

In [2]:
#import tensorflow and setting seed value
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In [3]:
#Importing train and val CSV files and setting batch size as 15
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 15

In [4]:
#generator function with batch size=15, img_idx = [0,2,4....,18], image size = (100,100), also covered with remaining data points
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(0,30,2)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    im = imresize(image,(100,100))
                    im1 = np.asarray(im).astype('float32')
                    batch_data[folder,idx,:,:,0] = (im1[:,:,0]-im1[:,:,0].min())/(im1[:,:,0].max()-im1[:,:,0].min())
                    batch_data[folder,idx,:,:,1] = (im1[:,:,1]-im1[:,:,1].min())/(im1[:,:,1].max()-im1[:,:,1].min())
                    batch_data[folder,idx,:,:,2] = (im1[:,:,2]-im1[:,:,2].min())/(im1[:,:,2].max()-im1[:,:,2].min())
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        extra=int(len(folder_list))-(num_batches*batch_size)
        if extra>0:
            batch=num_batches
            batch_data = np.zeros((extra,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((extra,5)) # batch_labels is the one hot representation of the output
            for folder in range(extra): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    im = imresize(image,(100,100))
                    im1 = np.asarray(im).astype('float32')
                    batch_data[folder,idx,:,:,0] = (im1[:,:,0]-im1[:,:,0].min())/(im1[:,:,0].max()-im1[:,:,0].min())
                    batch_data[folder,idx,:,:,1] = (im1[:,:,1]-im1[:,:,1].min())/(im1[:,:,1].max()-im1[:,:,1].min())
                    batch_data[folder,idx,:,:,2] = (im1[:,:,2]-im1[:,:,2].min())/(im1[:,:,2].max()-im1[:,:,2].min())
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 35
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 35


In [6]:
#importing required libraries
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [7]:
#Model Building
model = Sequential()
model.add(Conv3D(8,kernel_size=(3, 3, 3), activation='relu' , input_shape=(15, 100, 100, 3) )) 
model.add(Conv3D(8,kernel_size=(3, 3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))

model.add(Conv3D(16,kernel_size=(3,3,3), activation='relu', padding='same'))  
model.add(Conv3D(16,kernel_size=(3,3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))

model.add(Conv3D(32,kernel_size=(3,3,3), activation='relu', padding='same'))  
model.add(Conv3D(32,kernel_size=(3,3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))

model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [8]:
#setting optimizer and them compiling the model
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 98, 98, 8)     656       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 98, 98, 8)     1736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 98, 98, 8)     32        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 49, 49, 8)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 6, 49, 49, 16)     3472      
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 6, 49, 49, 16)     6928      
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 49, 49, 16)     64        
__________

In [9]:
#Calling generator function for both train and test data
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='loss',
                                   factor=0.0001,
                                   cooldown=0,
                                   patience=5,
                                   min_lr=1e-4)
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
#Model Generation
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 15
Source path =  Project_data/train ; batch size = 15
Epoch 1/35
45/45 [==============================] - 45s 998ms/step - loss: 1.7593 - categorical_accuracy: 0.2978 - val_loss: 5.8642 - val_categorical_accuracy: 0.2476

Epoch 00001: saving model to model_init_2020-03-0111_08_09.236698/model-00001-1.75929-0.29778-5.86422-0.24762.h5
Epoch 2/35
45/45 [==============================] - 41s 910ms/step - loss: 1.4118 - categorical_accuracy: 0.4074 - val_loss: 1.1800 - val_categorical_accuracy: 0.5714

Epoch 00002: saving model to model_init_2020-03-0111_08_09.236698/model-00002-1.41181-0.40741-1.18003-0.57143.h5
Epoch 3/35
45/45 [==============================] - 41s 921ms/step - loss: 1.2314 - categorical_accuracy: 0.5348 - val_loss: 1.1831 - val_categorical_accuracy: 0.5048

Epoch 00003: saving model to model_init_2020-03-0111_08_09.236698/model-00003-1.23142-0.53481-1.18308-0.50476.h5
Epoch 4/35
45/45 [==============================] - 42s

45/45 [==============================] - 40s 896ms/step - loss: 0.1442 - categorical_accuracy: 0.9556 - val_loss: 1.0057 - val_categorical_accuracy: 0.6190

Epoch 00030: saving model to model_init_2020-03-0111_08_09.236698/model-00030-0.14423-0.95556-1.00572-0.61905.h5
Epoch 31/35
45/45 [==============================] - 41s 921ms/step - loss: 0.1176 - categorical_accuracy: 0.9793 - val_loss: 1.1551 - val_categorical_accuracy: 0.5524

Epoch 00031: saving model to model_init_2020-03-0111_08_09.236698/model-00031-0.11763-0.97926-1.15507-0.55238.h5
Epoch 32/35
45/45 [==============================] - 42s 930ms/step - loss: 0.1126 - categorical_accuracy: 0.9748 - val_loss: 1.1565 - val_categorical_accuracy: 0.7048

Epoch 00032: saving model to model_init_2020-03-0111_08_09.236698/model-00032-0.11257-0.97481-1.15649-0.70476.h5
Epoch 33/35
45/45 [==============================] - 41s 910ms/step - loss: 0.0774 - categorical_accuracy: 0.9867 - val_loss: 0.9198 - val_categorical_accuracy: 0.752

In [ ]:
# We got best model at 28 epoch with 80% test accuracy and 95% train accuracy

In [ ]:
# Below is the LSTM Model we have tried which results in 77% test accuracy and 98% train accuracy
# model_2D = Sequential()

# model_2D.add(Conv2D(8, (3, 3), padding='same', input_shape=(100,100,3)))
# model_2D.add(Activation('relu')) 
# model_2D.add(BatchNormalization())
# model_2D.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

# model_2D.add(Conv2D(16, (3, 3), padding='same'))
# model_2D.add(Activation('relu')) 
# model_2D.add(BatchNormalization())
# model_2D.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

# model_2D.add(Conv2D(32, (3, 3), padding='same'))
# model_2D.add(Activation('relu'))
# model_2D.add(BatchNormalization())
# model_2D.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

# model_2D.add(Conv2D(64, (3, 3), padding='same'))
# model_2D.add(Activation('relu'))
# model_2D.add(BatchNormalization())
# model_2D.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

# model_2D.add(Dropout(0.5))
# model_2D.add(Flatten())

# model_2D.add(Dense(64, activation='relu'))
# model_2D.add(BatchNormalization())
# model_2D.add(Dropout(0.5))

# model_TD = Sequential()
# model_TD.add(TimeDistributed(model_2D, input_shape=(15, 100, 100,3)))
# model_TD.add(LSTM(64))
# model_TD.add(Dense(5, activation='softmax'))